In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
TRAIN_DIR = '/content/drive/MyDrive/dataset'
TEST_DIR  = '/content/drive/MyDrive/test'

IMG_SIZE = (224, 224)
BATCH_SIZE = 8
SEED = 42
NUM_CLASSES = 5

In [4]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

In [5]:
train_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=SEED
)

Found 200 images belonging to 5 classes.


In [6]:
val_data = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=SEED
)

Found 50 images belonging to 5 classes.


In [7]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.4)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 23,597,957 (90.02 MB)

 Trainable params: 23,544,837 (89.82 MB)

 Non-trainable params: 53,120 (207.50 KB)

In [8]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
]

In [9]:
base_model.trainable = False

model.compile(
    optimizer=Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
history1 = model.fit(
    train_data,
    epochs=20,
    validation_data=val_data,
    callbacks=callbacks
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
 6/25 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.1750 - loss: 2.6015

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (91586966 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2305 - loss: 2.4752

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:3452: DecompressionBombWarning: Image size (93801708 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.2355 - loss: 2.4610 - val_accuracy: 0.6000 - val_loss: 1.1051 - learning_rate: 0.0010
Epoch 2/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 70s 3s/step - accuracy: 0.6302 - loss: 0.9651 - val_accuracy: 0.7400 - val_loss: 0.8479 - learning_rate: 0.0010
Epoch 3/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.7640 - loss: 0.7284 - val_accuracy: 0.7200 - val_loss: 0.7541 - learning_rate: 0.0010
Epoch 4/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 68s 3s/step - accuracy: 0.7744 - loss: 0.4781 - val_accuracy: 0.7000 - val_loss: 0.8949 - learning_rate: 0.0010
Epoch 5/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.8257 - loss: 0.4023 - val_accuracy: 0.7400 - val_loss: 1.0095 - learning_rate: 0.0010
Epoch 6/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.8574 - loss: 0.3782 - val_accuracy: 0.7600 - val_loss: 0.9948 - learning_rate: 0.0010
Epoch 7/20
25/25 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - accuracy: 0.9042 - loss: 0.3316 - val_accuracy: 0.7200 - va

In [11]:
base_model.trainable = True
for layer in base_model.layers[:-30]:
    layer.trainable = False

model.compile(
    optimizer=Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
history2 = model.fit(
    train_data,
    epochs=40,
    validation_data=val_data,
    callbacks=callbacks
)

Epoch 1/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 106s 4s/step - accuracy: 0.7521 - loss: 0.7246 - val_accuracy: 0.7200 - val_loss: 0.8588 - learning_rate: 1.0000e-05
Epoch 2/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.8447 - loss: 0.4863 - val_accuracy: 0.7600 - val_loss: 0.8346 - learning_rate: 1.0000e-05
Epoch 3/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.8058 - loss: 0.4902 - val_accuracy: 0.7600 - val_loss: 0.9034 - learning_rate: 1.0000e-05
Epoch 4/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 90s 4s/step - accuracy: 0.8385 - loss: 0.3981 - val_accuracy: 0.7400 - val_loss: 1.0342 - learning_rate: 1.0000e-05
Epoch 5/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.8584 - loss: 0.4967 - val_accuracy: 0.7800 - val_loss: 0.9606 - learning_rate: 1.0000e-05
Epoch 6/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 85s 3s/step - accuracy: 0.9104 - loss: 0.3277 - val_accuracy: 0.7400 - val_loss: 0.9624 - learning_rate: 5.0000e-06
Epoch 7/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 89s 4s/step - accuracy: 0.7925 - loss

In [13]:
val_loss, val_acc = model.evaluate(val_data)
print("Validation Accuracy:", val_acc)

7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.8124 - loss: 0.6739
Validation Accuracy: 0.7599999904632568


In [14]:
train_acc_2 = history2.history['accuracy'][-1]
val_acc_2   = history2.history['val_accuracy'][-1]

print("Train Accuracy:", round(train_acc_2,2))
print("Val Accuracy:", round(val_acc_2,2))

Train Accuracy: 0.86
Val Accuracy: 0.76


# TES

In [18]:
import os
if os.path.exists(TEST_DIR):
    test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

    test_data = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
    test_loss, test_acc = model.evaluate(test_data)
    print("Test Accuracy:", round(test_acc,2))

Found 50 images belonging to 5 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.7361 - loss: 0.7373
Test Accuracy: 0.76


In [19]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_true = test_data.classes
y_pred = np.argmax(model.predict(test_data), axis=1)

print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))
print("\nClassification Report:\n",
      classification_report(y_true, y_pred, target_names=list(test_data.class_indices.keys())))

7/7 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step
Confusion Matrix:
 [[7 0 0 1 2]
 [2 7 0 1 0]
 [0 0 9 0 1]
 [1 0 1 8 0]
 [2 0 0 1 7]]

Classification Report:
               precision    recall  f1-score   support

      kawung       0.58      0.70      0.64        10
 megamendung       1.00      0.70      0.82        10
      parang       0.90      0.90      0.90        10
   sidomukti       0.73      0.80      0.76        10
     truntum       0.70      0.70      0.70        10

    accuracy                           0.76        50
   macro avg       0.78      0.76      0.76        50
weighted avg       0.78      0.76      0.76        50



In [20]:
import time
import numpy as np

# pastikan test_data.shuffle = False
num_images = test_data.samples

start_time = time.time()

_ = model.predict(test_data, verbose=0)

end_time = time.time()

total_time = end_time - start_time
avg_inference_time = (total_time / num_images) * 1000  # ms per image

print(f"Average Inference Time: {avg_inference_time:.2f} ms/image")

Average Inference Time: 339.24 ms/image
